In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
forest_mod = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Combined_CoverType_original_data2.csv')

In [0]:
forest_mod = forest_mod.drop(columns = forest_mod.columns[0:1])

In [0]:
forest_mod['Wilderness'] = forest_mod['Wilderness'].astype('category')
forest_mod['Soil'] = forest_mod['Soil'].astype('category')
forest_mod['CoverType'] = forest_mod['CoverType'].astype('category')

In [7]:
forest_mod.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness,Soil,CoverType
0,2596,51,3,258,0,510,221,232,148,6279,1,29,4
1,2590,56,2,212,-6,390,220,235,151,6225,1,29,4
2,2804,139,9,268,65,3180,234,238,135,6121,1,12,2
3,2785,155,18,242,118,3090,238,238,122,6211,1,30,2
4,2595,45,2,153,-1,391,220,234,150,6172,1,29,4


In [0]:
forest_mod['Vertical_Scaled'] = forest_mod['Vertical_Distance_To_Hydrology'] - min(forest_mod['Vertical_Distance_To_Hydrology'])

In [0]:
forest_mod['Hydr_Dist'] = (forest_mod['Horizontal_Distance_To_Hydrology']**2 + forest_mod['Vertical_Distance_To_Hydrology']**2)**0.5

In [0]:
forest_mod['Hillshades'] = (forest_mod['Hillshade_3pm'] + forest_mod['Hillshade_Noon'] + forest_mod['Hillshade_9am'])/3

In [0]:
forest_mod['Fire_Road'] = forest_mod['Horizontal_Distance_To_Fire_Points'] + forest_mod['Horizontal_Distance_To_Roadways']

In [0]:
forest_mod['Avg_Horizontal'] = (forest_mod['Horizontal_Distance_To_Fire_Points'] + 
                              forest_mod['Horizontal_Distance_To_Hydrology'] + 
                              forest_mod['Horizontal_Distance_To_Roadways'])/3

In [0]:
forest_mod['Fire_Hydr'] = (forest_mod['Horizontal_Distance_To_Fire_Points'] + forest_mod['Horizontal_Distance_To_Hydrology'])/2

In [0]:
from math import degrees

In [0]:
forest_mod['Hydr_Slope'] = forest_mod['Vertical_Distance_To_Hydrology']/forest_mod['Horizontal_Distance_To_Hydrology']

In [0]:
a = []
for i in forest_mod['Hydr_Slope']:
  a.append(degrees(np.arctan(i)))

In [0]:
b = pd.DataFrame({'a':a})
forest_mod['Hydr_Slope'] = np.where(b['a'].isnull(),0,b['a'])
forest_mod['Hydr_Slope'].isnull().sum()

0

In [0]:
forest_mod.to_csv('/content/drive/My Drive/Colab Notebooks/Combined_CoverType_Feat_Add.csv')

### Logistic Regression

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [0]:
x = forest_mod.drop(columns = ['CoverType'])
y = forest_mod['CoverType']

In [0]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .3, random_state=25)

In [158]:
LogReg = LogisticRegression()
LogReg.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
y_testpred = LogReg.predict(x_test)

In [160]:
print('Training Accuracy for Logistic Regression: ', accuracy_score(y_train,LogReg.predict(x_train)))

Training Accuracy for Logistic Regression:  0.6257388593290518


In [161]:
print('Training Accuracy for Logistic Regression: ', accuracy_score(y_test,y_testpred))

Training Accuracy for Logistic Regression:  0.6272432072700569


In [162]:
print(classification_report(y_test, y_testpred))

              precision    recall  f1-score   support

           1       0.63      0.61      0.62     63514
           2       0.63      0.79      0.70     85045
           3       0.59      0.29      0.39     10747
           4       0.26      0.01      0.01     14998

    accuracy                           0.63    174304
   macro avg       0.53      0.42      0.43    174304
weighted avg       0.59      0.63      0.59    174304



In [163]:
print(confusion_matrix(y_test,y_testpred))

[[38791 24541   108    74]
 [16749 67321   811   164]
 [  111  7492  3131    13]
 [ 5814  7809  1287    88]]


In [0]:
forest_mod = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Combined_CoverType_Feat_Add.csv')

In [5]:
forest_mod.head()

,Unnamed: 0,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness,Soil,CoverType,Vertical_Scaled,Hydr_Dist,Hillshades,Fire_Road,Avg_Horizontal,Fire_Hydr,Hydr_Slope
0,0,2596,51,3,258,0,510,221,232,148,6279,1,29,4,173,258.000000,200.333333,6789,2349.000000,3268.5,0.000000
1,1,2590,56,2,212,-6,390,220,235,151,6225,1,29,4,167,212.084889,202.000000,6615,2275.666667,3218.5,-1.621146
2,2,2804,139,9,268,65,3180,234,238,135,6121,1,12,2,238,275.769832,202.333333,9301,3189.666667,3194.5,13.633114
3,3,2785,155,18,242,118,3090,238,238,122,6211,1,30,2,291,269.235956,199.333333,9301,3181.000000,3226.5,25.994016
4,4,2595,45,2,153,-1,391,220,234,150,6172,1,29,4,172,153.003268,201.333333,6563,2238.666667,3162.5,-0.374477


In [0]:
forest_mod = forest_mod.drop(columns = forest_mod.columns[0:1])

In [0]:
forest_mod['Wilderness'] = forest_mod['Wilderness'].astype('category')
forest_mod['Soil'] = forest_mod['Soil'].astype('category')
forest_mod['CoverType'] = forest_mod['CoverType'].astype('category')

### Decision Tree

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.tree import DecisionTreeClassifier
import warnings
warnings.filterwarnings("ignore")

In [0]:
x = forest_mod.drop(columns = ['CoverType'])
y = forest_mod['CoverType']

In [0]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .3, random_state=25)

In [10]:
dt = DecisionTreeClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
dt.fit(x_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=19, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=25, splitter='best')

In [0]:
y_testpred = dt.predict(x_test)

In [12]:
print('Training Accuracy for DecisionTree Classifier: ', accuracy_score(y_train,dt.predict(x_train)))

Training Accuracy for DecisionTree Classifier:  0.9332567837367349


In [13]:
print('Training Accuracy for DecisionTree Classifier: ', accuracy_score(y_test,y_testpred))

Training Accuracy for DecisionTree Classifier:  0.9070589315219387


In [14]:
print(classification_report(y_test, y_testpred))

              precision    recall  f1-score   support

           1       0.91      0.89      0.90     63514
           2       0.91      0.93      0.92     85045
           3       0.90      0.91      0.91     10747
           4       0.89      0.85      0.87     14998

    accuracy                           0.91    174304
   macro avg       0.90      0.90      0.90    174304
weighted avg       0.91      0.91      0.91    174304



In [15]:
print(confusion_matrix(y_test,y_testpred))

[[56785  6371     8   350]
 [ 5259 78812   286   688]
 [    0   377  9821   549]
 [  590   976   746 12686]]


In [16]:
pd.DataFrame({'columns':x.columns,'Importance':dt.feature_importances_})

,columns,Importance
0,Elevation,0.229007
1,Aspect,0.014209
2,Slope,0.009090
3,Horizontal_Distance_To_Hydrology,0.014819
4,Vertical_Distance_To_Hydrology,0.008680
5,Horizontal_Distance_To_Roadways,0.078766
6,Hillshade_9am,0.020986
7,Hillshade_Noon,0.017667
8,Hillshade_3pm,0.010484
9,Horizontal_Distance_To_Fire_Points,0.059252


### Random Forest

In [0]:
from sklearn.ensemble import RandomForestClassifier

In [18]:
rforest = RandomForestClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
rforest.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=19, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=25, verbose=0,
                       warm_start=False)

In [0]:
y_testpred = rforest.predict(x_test)

In [20]:
print('Training Accuracy for Random Forest Classifier: ', accuracy_score(y_train,rforest.predict(x_train)))

Training Accuracy for Random Forest Classifier:  0.9616530778838872


In [21]:
print('Training Accuracy for Random Forest Classifier: ', accuracy_score(y_test,y_testpred))

Training Accuracy for Random Forest Classifier:  0.9403054433633192


In [22]:
print(classification_report(y_test, y_testpred))

              precision    recall  f1-score   support

           1       0.95      0.92      0.94     63514
           2       0.93      0.96      0.95     85045
           3       0.94      0.95      0.95     10747
           4       0.95      0.87      0.91     14998

    accuracy                           0.94    174304
   macro avg       0.94      0.93      0.93    174304
weighted avg       0.94      0.94      0.94    174304



In [23]:
print(confusion_matrix(y_test,y_testpred))

[[58534  4837     1   142]
 [ 2574 82039   178   254]
 [    1   194 10249   303]
 [  461   951   509 13077]]


In [24]:
pd.DataFrame({'columns':x.columns,'Importance':rforest.feature_importances_})

,columns,Importance
0,Elevation,0.255239
1,Aspect,0.021902
2,Slope,0.011603
3,Horizontal_Distance_To_Hydrology,0.021159
4,Vertical_Distance_To_Hydrology,0.017692
5,Horizontal_Distance_To_Roadways,0.066006
6,Hillshade_9am,0.019600
7,Hillshade_Noon,0.017167
8,Hillshade_3pm,0.016306
9,Horizontal_Distance_To_Fire_Points,0.049572


### KNN

In [146]:
from sklearn.preprocessing import StandardScaler

x1 = forest_mod.drop(columns= ['CoverType','Soil','Wilderness'])
y_ = forest_mod['CoverType']

x_std = StandardScaler().fit_transform(x1)

x_std = pd.DataFrame(x_std, columns = x1.columns)

x_ = pd.concat([x_std,forest_mod[['Soil','Wilderness']]], axis = 1)
x_.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Vertical_Scaled,Hydr_Dist,Hillshades,Fire_Road,Avg_Horizontal,Fire_Hydr,Hydr_Slope,Soil,Wilderness
0,-1.297805,-0.935157,-1.482820,-0.053767,-0.796273,-1.180146,0.330743,0.439143,0.142960,3.246283,-0.796273,-0.083233,0.530152,1.043224,1.027143,3.171097,-1.041587,29,1
1,-1.319235,-0.890480,-1.616363,-0.270188,-0.899197,-1.257106,0.293388,0.590899,0.221342,3.205504,-0.899197,-0.294777,0.645367,0.969391,0.934802,3.097132,-1.228740,29,1
2,-0.554907,-0.148836,-0.681563,-0.006719,0.318742,0.532212,0.816364,0.742654,-0.196691,3.126965,0.318742,-0.001362,0.668410,2.109122,2.085710,3.061629,0.532279,12,1
3,-0.622768,-0.005869,0.520322,-0.129044,1.227908,0.474492,0.965786,0.742654,-0.536343,3.194931,1.227908,-0.031466,0.461022,2.109122,2.074797,3.108966,1.959275,30,1
4,-1.301377,-0.988770,-1.616363,-0.547771,-0.813427,-1.256464,0.293388,0.540313,0.195215,3.165479,-0.813427,-0.566983,0.599281,0.947327,0.888212,3.014291,-1.084819,29,1


In [0]:
x_train, x_test, y_train, y_test = train_test_split(x_, y_, test_size = .3, random_state=25)

In [0]:
from sklearn.neighbors import KNeighborsClassifier

In [148]:
knn = KNeighborsClassifier(metric = 'cityblock', n_neighbors= 4, weights= 'distance')
knn.fit(x_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='cityblock',
                     metric_params=None, n_jobs=None, n_neighbors=4, p=2,
                     weights='distance')

In [0]:
y_testpred = knn.predict(x_test)

In [150]:
print('Training Accuracy for KNN Classifier: ', accuracy_score(y_train,knn.predict(x_train)))

Training Accuracy for KNN Classifier:  1.0


In [151]:
print('Training Accuracy for KNN Classifier: ', accuracy_score(y_test,y_testpred))

Training Accuracy for KNN Classifier:  0.947775151459519


In [152]:
print(classification_report(y_test, y_testpred))

              precision    recall  f1-score   support

           1       0.95      0.94      0.95     63514
           2       0.95      0.96      0.96     85045
           3       0.93      0.93      0.93     10747
           4       0.91      0.91      0.91     14998

    accuracy                           0.95    174304
   macro avg       0.94      0.94      0.94    174304
weighted avg       0.95      0.95      0.95    174304



In [153]:
print(confusion_matrix(y_test,y_testpred))

[[60019  3205     2   288]
 [ 2717 81587   200   541]
 [    7   210  9988   542]
 [  312   568   511 13607]]


## Ensemble

In [0]:
#base_LR = LogisticRegression(random_state = 25)
base_DT = DecisionTreeClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
#base_NB = GaussianNB()
base_RF = RandomForestClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
base_KNN = KNeighborsClassifier(metric = 'cityblock', 
                                n_neighbors= 4, 
                                weights= 'distance')

In [0]:
x = forest_mod.drop(columns= ['CoverType'])
y = forest_mod['CoverType']

Bagging Classifier

In [0]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .3, random_state=25)

In [0]:
from sklearn.ensemble import BaggingClassifier

Bagged DT

In [30]:
bgcl_DT = BaggingClassifier(base_estimator=base_DT, n_estimators = 51, oob_score=True, random_state=25)

bgcl_DT.fit(x_train, y_train)



BaggingClassifier(base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='entropy',
                                                        max_depth=19,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=10,
                                                        min_samples_split=10,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort='deprecated',
                                                        random_state=25,

In [0]:
y_testpred = bgcl_DT.predict(x_test)

In [32]:
print('Training Accuracy for Bagged DT: ', accuracy_score(y_train,bgcl_DT.predict(x_train)))

Training Accuracy for Bagged DT:  0.9670574466201796


In [33]:
print('Training Accuracy for Bagged DT: ', accuracy_score(y_test,y_testpred))

Training Accuracy for Bagged DT:  0.9458933816779879


In [34]:
print('OOB Score for Bagged DT:', bgcl_DT.oob_score_)

OOB Score for Bagged DT: 0.9419608170972786


In [35]:
print(classification_report(y_test, y_testpred))

              precision    recall  f1-score   support

           1       0.95      0.93      0.94     63514
           2       0.94      0.96      0.95     85045
           3       0.94      0.96      0.95     10747
           4       0.95      0.89      0.92     14998

    accuracy                           0.95    174304
   macro avg       0.95      0.94      0.94    174304
weighted avg       0.95      0.95      0.95    174304



In [36]:
print(confusion_matrix(y_test,y_testpred))

[[59260  4078     4   172]
 [ 2591 81980   178   296]
 [    0   180 10277   290]
 [  420   767   455 13356]]


Bagged RF

In [37]:
bgcl_RF = BaggingClassifier(base_estimator=base_RF, n_estimators = 51, oob_score=True, random_state=25)
bgcl_RF.fit(x_train, y_train)

BaggingClassifier(base_estimator=RandomForestClassifier(bootstrap=True,
                                                        ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='entropy',
                                                        max_depth=19,
                                                        max_features='auto',
                                                        max_leaf_nodes=None,
                                                        max_samples=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=10,
                                                        min_samples_split=10,
                                                        min_weight_fraction_leaf=0.0,
  

In [0]:
y_testpred = bgcl_RF.predict(x_test)

In [39]:
print('Training Accuracy for Bagged RF: ', accuracy_score(y_train,bgcl_RF.predict(x_train)))

Training Accuracy for Bagged RF:  0.9458284567798028


In [40]:
print('Training Accuracy for Bagged RF: ', accuracy_score(y_test,y_testpred))

Training Accuracy for Bagged RF:  0.9275576005140445


In [41]:
print('OOB Score for Bagged RF:', bgcl_RF.oob_score_)

OOB Score for Bagged RF: 0.9270213519281647


In [42]:
print(classification_report(y_test, y_testpred))

              precision    recall  f1-score   support

           1       0.94      0.91      0.92     63514
           2       0.92      0.96      0.94     85045
           3       0.92      0.94      0.93     10747
           4       0.94      0.83      0.88     14998

    accuracy                           0.93    174304
   macro avg       0.93      0.91      0.92    174304
weighted avg       0.93      0.93      0.93    174304



In [43]:
print(confusion_matrix(y_test,y_testpred))

[[57709  5630     0   175]
 [ 3158 81388   226   273]
 [    3   276 10111   357]
 [  630  1283   616 12469]]
[[57709  5630     0   175]
 [ 3158 81388   226   273]
 [    3   276 10111   357]
 [  630  1283   616 12469]]


Bagged KNN

In [21]:
bgcl_KNN = BaggingClassifier(base_estimator=base_KNN, n_estimators = 51, oob_score=True, random_state=25)
bgcl_KNN.fit(x_train, y_train)

BaggingClassifier(base_estimator=KNeighborsClassifier(algorithm='auto',
                                                      leaf_size=30,
                                                      metric='cityblock',
                                                      metric_params=None,
                                                      n_jobs=None,
                                                      n_neighbors=4, p=2,
                                                      weights='distance'),
                  bootstrap=True, bootstrap_features=False, max_features=1.0,
                  max_samples=1.0, n_estimators=51, n_jobs=None, oob_score=True,
                  random_state=25, verbose=0, warm_start=False)

In [0]:
y_testpred = bgcl_KNN.predict(x_test)

In [23]:
print('Training Accuracy for Bagged KNN: ', accuracy_score(y_train,bgcl_KNN.predict(x_train)))

Training Accuracy for Bagged KNN:  0.9999803298681118


In [24]:
print('Training Accuracy for Bagged KNN: ', accuracy_score(y_test,y_testpred))

Training Accuracy for Bagged KNN:  0.9690024325316688


In [25]:
print('OOB Score for Bagged KNN:', bgcl_KNN.oob_score_)

OOB Score for Bagged KNN: 0.9669517196612804


In [26]:
print(classification_report(y_test, y_testpred))

              precision    recall  f1-score   support

           1       0.97      0.97      0.97     63514
           2       0.97      0.98      0.97     85045
           3       0.97      0.96      0.96     10747
           4       0.95      0.94      0.95     14998

    accuracy                           0.97    174304
   macro avg       0.96      0.96      0.96    174304
weighted avg       0.97      0.97      0.97    174304



In [27]:
print(confusion_matrix(y_test,y_testpred))

[[61472  1875     6   161]
 [ 1695 82928   116   306]
 [    3   133 10356   255]
 [  206   401   246 14145]]
[[61472  1875     6   161]
 [ 1695 82928   116   306]
 [    3   133 10356   255]
 [  206   401   246 14145]]
[[61472  1875     6   161]
 [ 1695 82928   116   306]
 [    3   133 10356   255]
 [  206   401   246 14145]]
[[61472  1875     6   161]
 [ 1695 82928   116   306]
 [    3   133 10356   255]
 [  206   401   246 14145]]
[[61472  1875     6   161]
 [ 1695 82928   116   306]
 [    3   133 10356   255]
 [  206   401   246 14145]]
[[61472  1875     6   161]
 [ 1695 82928   116   306]
 [    3   133 10356   255]
 [  206   401   246 14145]]


AdaBoost

In [0]:
from sklearn.ensemble import AdaBoostClassifier

AdaBoost DT

In [0]:
abcl_DT = AdaBoostClassifier(base_estimator=base_DT,n_estimators=50,random_state=25)

In [33]:
abcl_DT.fit(x_train,y_train)

AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                         class_weight=None,
                                                         criterion='entropy',
                                                         max_depth=19,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=10,
                                                         min_samples_split=10,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort='deprecated',
                    

In [0]:
y_testpred = abcl_DT.predict(x_test)

In [35]:
print('Training Accuracy for AdaBoost DT: ', accuracy_score(y_train,abcl_DT.predict(x_train)))

Training Accuracy for AdaBoost DT:  0.9999975412335139


In [36]:
print('Training Accuracy for AdaBoost DT: ', accuracy_score(y_test,y_testpred))

Training Accuracy for AdaBoost DT:  0.955818569854966


In [37]:
print(classification_report(y_test, y_testpred))

              precision    recall  f1-score   support

           1       0.96      0.95      0.95     63514
           2       0.95      0.97      0.96     85045
           3       0.96      0.96      0.96     10747
           4       0.96      0.91      0.93     14998

    accuracy                           0.96    174304
   macro avg       0.96      0.95      0.95    174304
weighted avg       0.96      0.96      0.96    174304



In [38]:
print(confusion_matrix(y_test,y_testpred))

[[60125  3278     0   111]
 [ 2212 82532   121   180]
 [    3   142 10319   283]
 [  370   682   319 13627]]


Need to run from here

AdaBoost RF

In [0]:
abcl_RF = AdaBoostClassifier(base_estimator=base_RF,n_estimators=50,random_state=25)

In [20]:
abcl_RF.fit(x_train,y_train)

AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=RandomForestClassifier(bootstrap=True,
                                                         ccp_alpha=0.0,
                                                         class_weight=None,
                                                         criterion='entropy',
                                                         max_depth=19,
                                                         max_features='auto',
                                                         max_leaf_nodes=None,
                                                         max_samples=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=10,
                                                         min_samples_split=10,
                                    

In [0]:
y_testpred = abcl_RF.predict(x_test)

In [22]:
print('Training Accuracy for AdaBoost RF: ', accuracy_score(y_train,abcl_RF.predict(x_train)))

Training Accuracy for AdaBoost RF:  0.9269549652330419


In [23]:
print('Training Accuracy for AdaBoost RF: ', accuracy_score(y_test,y_testpred))

Training Accuracy for AdaBoost RF:  0.8896927207637232


In [24]:
print(classification_report(y_test, y_testpred))

              precision    recall  f1-score   support

           1       0.90      0.83      0.87     63514
           2       0.87      0.95      0.91     85045
           3       0.92      0.94      0.93     10747
           4       0.94      0.77      0.85     14998

    accuracy                           0.89    174304
   macro avg       0.91      0.87      0.89    174304
weighted avg       0.89      0.89      0.89    174304



In [25]:
print(confusion_matrix(y_test,y_testpred))

[[53020 10433     0    61]
 [ 4278 80402   167   198]
 [    1   174 10089   483]
 [ 1771   964   697 11566]]
[[53020 10433     0    61]
 [ 4278 80402   167   198]
 [    1   174 10089   483]
 [ 1771   964   697 11566]]


GradientBoost

In [0]:
from sklearn.ensemble import GradientBoostingClassifier

In [27]:
gbcl = GradientBoostingClassifier(random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10,
                                  n_estimators = 50)
gbcl.fit(x_train, y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=19,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=10, min_samples_split=10,
                           min_weight_fraction_leaf=0.0, n_estimators=50,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=25, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [0]:
y_testpred = gbcl.predict(x_test)

In [29]:
print('Training Accuracy for GradientBoost Classifier: ', accuracy_score(y_train, gbcl.predict(x_train)))

Training Accuracy for GradientBoost Classifier:  1.0


In [30]:
print('Training Accuracy for GradientBoost Classifier: ', accuracy_score(y_test,y_testpred))

Training Accuracy for GradientBoost Classifier:  0.9741772994308794


In [31]:
print(classification_report(y_test, y_testpred))

              precision    recall  f1-score   support

           1       0.98      0.97      0.97     63514
           2       0.97      0.98      0.98     85045
           3       0.97      0.97      0.97     10747
           4       0.96      0.94      0.95     14998

    accuracy                           0.97    174304
   macro avg       0.97      0.97      0.97    174304
weighted avg       0.97      0.97      0.97    174304



In [32]:
print(confusion_matrix(y_test,y_testpred))

[[61692  1704     1   117]
 [ 1172 83573   101   199]
 [    1   116 10426   204]
 [  220   410   256 14112]]
